# Imports

In [ ]:
import torch
import torch.nn as nn
from utils.datautils import *
from utils.MLutils import *
from utils.resources import *
from transformers import BertTokenizerFast
from sklearn.model_selection import train_test_split
from transformers import BertModel
import unicodedata

# linea que arregla algunos errores de loadeo de datasets
# pip install --upgrade datasets

/home/fabro/Documents/uba/aprendizaje_automatico/Puntualizacion-Capitalizacion/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Procesamiento

In [4]:
linux = True
device = None

if linux:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
else:
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

print("usando:", device)

usando: cuda


# Ejericio b)

## Busqueda de fuentes

### Fuente 1: Conjunto de preguntas en espa;ol

In [5]:
questions, question_for_mixture = get_questions()

Se descargaron 5000 preguntas en Español.


### Fuente 2: Dataset provisto para Notebook 10

In [6]:
oraciones_rnn = get_notebook_dataset()

Se descargaron 997 oraciones en Español (del dataset del notebook 10).


### Fuente 3: Dataset sintetico generado con Gemini

In [7]:
oraciones_sinteticas = get_gemini_dataset()

Hay 1413 oraciones sintéticas.


### Fuente 4: Articulos de Wikipedia

In [8]:
frases_wikipedia = get_wikipedia_dataset()

['Argentina, oficialmente República Argentina,[a]\u200b es un país soberano de América del Sur, ubicado en el extremo sur y sudeste de ese subcontinente.', 'Adopta la forma de gobierno republicana, democrática, representativa y federal.', 'Poseen Carta Magna, bandera y fuerzas de seguridad propias, el dominio de los recursos naturales circunscriptos en su territorio y delegan los poderes exclusivos al Gobierno Federal.', 'Hasta mediados del siglo XX, fue una de las economías más prósperas del mundo.', 'No obstante, es la segunda economía más importante de Sudamérica —detrás de Brasil— y la 24.º más grande del mundo por PIB nominal.']


### Fuente 5: Subtitulos de peliculas

In [9]:
esperando_la_carroza, frases_relatos_salvajes = get_pelis_dataset()

✅ Se extrajeron 947 frases completas y se guardaron en 'dialogos_esperando_la_carroza.json'
✅ Frases extraídas y guardadas. Total: 947
['No, es la televisión...', '¡Gran ciencia cambiar un pañal! Al final...', 'Por empezar dejá de hinchar... Ay, doña Elisa, no, no era por usted. Cómo... Oiga, le habla Elvira: ¿qué pasa con el agua? Hice ravioles, ¿quiere tragedia mayor? Ah, ¿usted también? ¡Qué casualidad! Bueno, mire, ¿me hace un favor? No la tire. Quiero decir, usted hierve sus ravioles y cuando estén listos me llama por teléfono y yo le mando a buscar el agua. Ay, gracias, es usted un amor. Yo hago puchero, ella hace puchero. Yo hago ravioles, ella hace ravioles, ¡Qué país!', 'Pero yo me acosté a las cuatro de la mañana.', 'Che: ¿por qué no ventilan los trapitos sucios en la azotea de su propia casa?', 'SI existe, pero doña Elisa. Yo no tengo la más mínima duda de que no existe otra cosa que el infierno.', 'Lo mandé a cambiar. Bastante me deprime comer los ravioles que hace ésta com

### Fuente 6 (beta): Mixture de oraciones

In [10]:
cant_oraciones = len(oraciones_sinteticas)
question_for_mixture = [re.sub(r'[\\\(\)!¡“]', '', unicodedata.normalize("NFC", q).strip()) for q in question_for_mixture]
oraciones_sinteticas = [re.sub(r'[\\\(\)!¡“]', '', unicodedata.normalize("NFC", a).strip()) for a in oraciones_sinteticas]

tanda_1 = question_for_mixture[:cant_oraciones]
question_affirmation = [f"{q} {a}" for q, a in zip(tanda_1, oraciones_sinteticas)]

tanda_2 = question_for_mixture[cant_oraciones:2*cant_oraciones]
affirmation_question = [f"{a} {q}" for q, a in zip(tanda_2, oraciones_sinteticas)]

tanda_3 = question_for_mixture[2*cant_oraciones:3*cant_oraciones]
tanda_3_shuffled = random.sample(tanda_3, len(tanda_3))
question_question = [f"{q} {p}" for q, p in zip(tanda_3, tanda_3_shuffled)]

mixtures = question_affirmation + affirmation_question + question_question

random.sample(mixtures, 5)


['¿Cuál es el estado civil de Pilar? La capital de Azerbaiyán, Bakú, es conocida como la ciudad de los vientos.',
 '¿Cómo se origina una enana blanca? ¿En qué se convertían ellas?',
 '¿A cuántos grados como máximo pueden estar las enanas blancas que podrían tener una área habitable? ¿Cuánto dinero se cree que se había recaudado con The Legend of Zelda: Ocarina of Time a finales de 1998 en América?',
 '¿Cómo se conoce en la actualidad África del Sudoeste? La consola Genesis de Sega era popular.',
 'La ciudad de Leptis Magna en Libia fue una ciudad romana. ¿Dónde está siendo atendido Bah?']

## Juntamos las fuentes

In [11]:
oraciones_raw = questions + oraciones_rnn + oraciones_sinteticas + frases_wikipedia + esperando_la_carroza  + frases_relatos_salvajes + mixtures

print('Cantidad total de oraciones:',len(oraciones_raw))
print('Cantidad de oraciones de preguntas:',len(questions))
print('Cantidad de oraciones en espa;ol de hugging face:',len(oraciones_rnn))
print('Cantidad de oraciones sintéticas:',len(oraciones_sinteticas))
print('Cantidad de oraciones de Wikipedia:',len(frases_wikipedia))
print('Cantidad de oraciones de Esperando la carroza:',len(esperando_la_carroza))
print('Cantidad de oraciones de Relatos Salvajes:',len(frases_relatos_salvajes))
print('Cantidad de oraciones Compuestas:',len(mixtures))

print("Algunas oraciones aleatorias:")
random.sample(oraciones_raw, 5)

Cantidad total de oraciones: 20244
Cantidad de oraciones de preguntas: 5000
Cantidad de oraciones en espa;ol de hugging face: 997
Cantidad de oraciones sintéticas: 1413
Cantidad de oraciones de Wikipedia: 6648
Cantidad de oraciones de Esperando la carroza: 947
Cantidad de oraciones de Relatos Salvajes: 1000
Cantidad de oraciones Compuestas: 4239
Algunas oraciones aleatorias:


['Bebí una Coca-Cola en el bar.',
 'Adobe ofrece descuentos para los estudiantes.',
 'Mi cámara digital es de la marca Canon. ¿Qué fenómeno meteorológico amenaza al litoral oriental norteamericano?',
 '¿Cuál es la función del INEGI?',
 '¿Cuáles son las características del clima mediterráneo continentalizado?']

Separamos en conjuntos de `train` y `test` con el tokenizer de `BERT`

In [12]:
tokenizer = BertTokenizerFast.from_pretrained("bert-base-multilingual-cased")

train_sents, test_sents = train_test_split(oraciones_raw, test_size=0.05, random_state=42)

dataloader_train = get_dataloader(oraciones_raw=oraciones_raw, max_length=64, batch_size=64, device=device, tokenizer=tokenizer)
dataloader_test = get_dataloader(oraciones_raw=test_sents, max_length=64, batch_size=64, device=device, tokenizer=tokenizer)

print(len(train_sents))
print(len(test_sents))

19231
1013


## Importamos el modelo

In [13]:
from train.PunctuationCapitalizationRNNBidirectional import PunctuationCapitalizationRNNBidirectional

model_name = "bert-base-multilingual-cased"
bert_model = BertModel.from_pretrained(model_name)

bert_embeddings = bert_model.embeddings.word_embeddings
for param in bert_model.parameters():
    param.requires_grad = False

N = 2
for layer in bert_model.encoder.layer[-N:]:
    for param in layer.parameters():
        param.requires_grad = True

for param in bert_model.pooler.parameters():
    param.requires_grad = True


model = PunctuationCapitalizationRNNBidirectional(
    bert_model = bert_model,
    hidden_dim=256,
    num_punct_classes=len(PUNCT_TAGS),
    num_cap_classes=len(CAP_TAGS)
).to(device)

ckpt = torch.load("model_bidirec.pt", map_location=device)
# si guardaste state_dict puro
if isinstance(ckpt, dict) and "model_state_dict" not in ckpt:
    model.load_state_dict(ckpt)

# si guardaste un dict con más cosas (epoch, optim, etc.)
elif "model_state_dict" in ckpt:
    model.load_state_dict(ckpt["model_state_dict"])

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

Total parameters: 180,944,905
Trainable parameters: 17,857,801


In [14]:
punct_weights_tensor, cap_weights_tensor = compute_class_weights(
    dataloader_train,
    num_punct_classes=len(PUNCT_TAGS),
    num_cap_classes=len(CAP_TAGS),
    device=device,
    beta=0.7
)

criterion_punct = nn.CrossEntropyLoss(ignore_index=-100, weight=punct_weights_tensor)
criterion_cap   = nn.CrossEntropyLoss(ignore_index=-100, weight=cap_weights_tensor)

trainable_params = [
    p for p in bert_model.parameters() if p.requires_grad
] + list(model.projection.parameters()) \
  + list(model.rnn.parameters()) \
  + list(model.punct_classifier.parameters()) \
  + list(model.cap_classifier.parameters())

optimizer = torch.optim.AdamW(trainable_params, lr=2e-5)

train(model, dataloader_train=dataloader_train, dataloader_test=dataloader_test,optimizer=optimizer, criterion_punct=criterion_punct, criterion_cap = criterion_cap, device=device, epochs=20)

Epoch 1 | Train Loss: 0.2961
Epoch 2 | Train Loss: 0.2634
Epoch 3 | Train Loss: 0.2460
Epoch 4 | Train Loss: 0.2331
Epoch 5 | Train Loss: 0.2244
Epoch 6 | Train Loss: 0.2134
Epoch 7 | Train Loss: 0.2084
Epoch 8 | Train Loss: 0.1982
Epoch 9 | Train Loss: 0.1921
Epoch 10 | Train Loss: 0.1851
Epoch 11 | Train Loss: 0.1798
Epoch 12 | Train Loss: 0.1729
Epoch 13 | Train Loss: 0.1673
Epoch 14 | Train Loss: 0.1605
Epoch 15 | Train Loss: 0.1561
Epoch 16 | Train Loss: 0.1516
Epoch 17 | Train Loss: 0.1474
Epoch 18 | Train Loss: 0.1418
Epoch 19 | Train Loss: 0.1377
Epoch 20 | Train Loss: 0.1340


## Evaluacion

In [ ]:
evaluate(model, dataloader_test, device)

In [36]:
entrada = "quiero aprender a programar en python conocés algún curso"
print(f"{predict_and_reconstruct(model, entrada, tokenizer, device, verbose=False)}")

Quiero aprender a programar en Python. Conocés algún curso.


## Export modelo

In [22]:
torch.save(model.state_dict(), "model_bidirec_more_data.pt")